In [1]:
import os, sys
# for accessing src, stan, etc.
sys.path.append(os.path.abspath(os.path.join("../..")))

import numpy as np
import tensorflow as tf
import pickle

from amortizer import amortizer
from src.models.HmmEam import constrain_parameters

from pandas import read_csv

/Users/simonkucharsky/projects/bayesflow/amortized-mixture/.venv/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 8)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 400, 3)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.
INFO:root:Loaded loss history from checkpoints/amortizer/history_100.pkl.
INFO:root:Networks loaded from checkpoints/amortizer/ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [2]:
def get_summary_conditions(subject):
    path = os.path.join('dutilh-resources', 'data', subject) + '.csv'
    df = read_csv(path)
    df = np.array(df)
    df = df[:400,...]

    rts = df[:,1:2].astype(np.float32)
    choice = np.abs(df[:,2]-3)
    responses = tf.one_hot(choice-1, 2)

    summary_conditions = tf.concat([rts, responses], axis=-1)
    summary_conditions = tf.expand_dims(summary_conditions, axis=0)

    return summary_conditions

def configure_data(subject):
    summary_conditions = get_summary_conditions(subject)

    output = {
        "posterior_inputs": {
            "summary_conditions": summary_conditions
        },
        "mixture_inputs": {
            "summary_conditions": summary_conditions
        }
    }
    return output

def sample(subject):
    input = configure_data(subject)
    unconstrained_posterior, classification = amortizer.sample(input, n_samples=4000)
    rts = input['posterior_inputs']['summary_conditions'][..., 0]
    posterior = constrain_parameters(unconstrained_posterior, rts)

    return posterior, classification

In [3]:
subjects = sorted(os.listdir(os.path.join('dutilh-resources', 'data')))
subjects = [os.path.splitext(s)[0] for s in subjects]

In [4]:
for subject in subjects:
    df = configure_data(subject)
    fit = sample(subject)

    with open(os.path.join('dutilh-resources', 'bayesflow-fits', subject) + '.pkl', 'wb') as f:
        pickle.dump(fit, f)